# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [1]:
# import libraries
import pandas as pd
import numpy as np
import sqlite3
import sqlalchemy
import matplotlib.pyplot as plt
import nltk

from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize

from nltk.stem import WordNetLemmatizer
from sklearn.model_selection import GridSearchCV
from sklearn.datasets import make_multilabel_classification
from sklearn.multioutput import MultiOutputClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report


%matplotlib inline

In [2]:
nltk.download(['punkt', 'wordnet'])

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


True

In [3]:
# load data from database
engine = sqlalchemy.create_engine('sqlite:///messages.db')
df = pd.read_sql_table('messages', con=engine)

In [4]:
df.head(3)

,id,message,original,genre,related,request,offer,aid_related,medical_help,medical_products,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,2,Weather update - a cold front from Cuba that c...,Un front froid se retrouve sur Cuba ce matin. ...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,7,Is the Hurricane over or is it not over,Cyclone nan fini osinon li pa fini,direct,1,0,0,1,0,0,...,0,0,1,0,1,0,0,0,0,0
2,8,Looking for someone but no name,"Patnm, di Maryani relem pou li banm nouvel li ...",direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [5]:
X = df['message']
Y = df.iloc[:, 4:]

### 2. Write a tokenization function to process your text data

In [6]:
def tokenize(text):
    tokenized = word_tokenize(text)
    lemmatizer = WordNetLemmatizer()
    
    cleaned = []
    for word in tokenized:
        cleaned.append(lemmatizer.lemmatize(word).lower().strip())
    return cleaned

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [7]:
pipeline = Pipeline([
    ('vect', CountVectorizer(tokenizer=tokenize)),
    ('tfidf', TfidfTransformer()),
    ('clf', MultiOutputClassifier(RandomForestClassifier()))
])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [8]:
X_train, X_test, y_train, y_test = train_test_split(X,Y)
pipeline.fit(X_train, y_train)

Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...oob_score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=1))])

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [9]:
y_pred = pipeline.predict(X_test)

In [10]:
for message, target in enumerate(y_test):
    print(target)
    print(classification_report(y_test[target], y_pred[:, message]))

related
             precision    recall  f1-score   support

          0       0.60      0.35      0.44      1516
          1       0.82      0.93      0.87      4984
          2       0.55      0.20      0.30        54

avg / total       0.77      0.79      0.77      6554

request
             precision    recall  f1-score   support

          0       0.88      0.98      0.93      5425
          1       0.84      0.37      0.52      1129

avg / total       0.88      0.88      0.86      6554

offer
             precision    recall  f1-score   support

          0       1.00      1.00      1.00      6528
          1       0.00      0.00      0.00        26

avg / total       0.99      1.00      0.99      6554

aid_related
             precision    recall  f1-score   support

          0       0.72      0.88      0.80      3832
          1       0.76      0.53      0.62      2722

avg / total       0.74      0.73      0.72      6554

medical_help
             precision    recall  f1-sco

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


### 6. Improve your model
Use grid search to find better parameters. 

In [11]:
pipeline.get_params() 

{'memory': None,
 'steps': [('vect',
   CountVectorizer(analyzer='word', binary=False, decode_error='strict',
           dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
           lowercase=True, max_df=1.0, max_features=None, min_df=1,
           ngram_range=(1, 1), preprocessor=None, stop_words=None,
           strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
           tokenizer=<function tokenize at 0x7fb1799a8950>, vocabulary=None)),
  ('tfidf',
   TfidfTransformer(norm='l2', smooth_idf=True, sublinear_tf=False, use_idf=True)),
  ('clf',
   MultiOutputClassifier(estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
               max_depth=None, max_features='auto', max_leaf_nodes=None,
               min_impurity_decrease=0.0, min_impurity_split=None,
               min_samples_leaf=1, min_samples_split=2,
               min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
               oob_score=False, random_state=None,

In [15]:
parameters = {
                'tfidf__use_idf': (True, False),
                'clf__estimator__max_depth': (10, 20, 30)
             }

In [16]:
cv = GridSearchCV(pipeline, param_grid=parameters)

In [17]:
cv.fit(X_train, y_train)
y_pred = cv.predict(X_test)

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [18]:
for message, target in enumerate(y_test):
    print(target)
    print(classification_report(y_test[target], y_pred[:, message]))

related
             precision    recall  f1-score   support

          0       0.00      0.00      0.00      1516
          1       0.76      1.00      0.86      4984
          2       0.00      0.00      0.00        54

avg / total       0.58      0.76      0.66      6554

request
             precision    recall  f1-score   support

          0       0.83      1.00      0.91      5425
          1       1.00      0.00      0.01      1129

avg / total       0.86      0.83      0.75      6554

offer
             precision    recall  f1-score   support

          0       1.00      1.00      1.00      6528
          1       0.00      0.00      0.00        26

avg / total       0.99      1.00      0.99      6554

aid_related
             precision    recall  f1-score   support

          0       0.62      0.98      0.76      3832
          1       0.85      0.16      0.27      2722

avg / total       0.72      0.64      0.56      6554

medical_help
             precision    recall  f1-sco

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

### Adaboost 

In [19]:
from sklearn.ensemble import AdaBoostClassifier, ExtraTreesClassifier
pipeline_alt_adaboost = Pipeline([
        ('vect', CountVectorizer(tokenizer=tokenize)),
        ('tfidf', TfidfTransformer()),
        ('clf', MultiOutputClassifier(AdaBoostClassifier()))
    ])

In [20]:
pipeline_alt_adaboost.fit(X_train, y_train)
y_pred = pipeline_alt_adaboost.predict(X_test)

In [21]:
for message, target in enumerate(y_test):
    print(target)
    print(classification_report(y_test[target], y_pred[:, message]))

related
             precision    recall  f1-score   support

          0       0.55      0.23      0.33      1516
          1       0.80      0.94      0.86      4984
          2       0.32      0.15      0.20        54

avg / total       0.74      0.77      0.73      6554

request
             precision    recall  f1-score   support

          0       0.91      0.97      0.94      5425
          1       0.78      0.53      0.63      1129

avg / total       0.89      0.89      0.89      6554

offer
             precision    recall  f1-score   support

          0       1.00      1.00      1.00      6528
          1       0.00      0.00      0.00        26

avg / total       0.99      0.99      0.99      6554

aid_related
             precision    recall  f1-score   support

          0       0.75      0.86      0.80      3832
          1       0.76      0.60      0.67      2722

avg / total       0.75      0.75      0.75      6554

medical_help
             precision    recall  f1-sco

### ExtraTrees

In [22]:
from sklearn.ensemble import AdaBoostClassifier, ExtraTreesClassifier
pipeline_alt_et = Pipeline([
        ('vect', CountVectorizer(tokenizer=tokenize)),
        ('tfidf', TfidfTransformer()),
        ('clf', MultiOutputClassifier(ExtraTreesClassifier()))
    ])

In [23]:
pipeline_alt_et.fit(X_train, y_train)
y_pred = pipeline_alt_et.predict(X_test)

In [24]:
for message, target in enumerate(y_test):
    print(target)
    print(classification_report(y_test[target], y_pred[:, message]))

related
             precision    recall  f1-score   support

          0       0.65      0.34      0.45      1516
          1       0.82      0.94      0.88      4984
          2       0.33      0.04      0.07        54

avg / total       0.78      0.80      0.77      6554

request
             precision    recall  f1-score   support

          0       0.88      0.98      0.93      5425
          1       0.83      0.35      0.49      1129

avg / total       0.87      0.88      0.85      6554

offer
             precision    recall  f1-score   support

          0       1.00      1.00      1.00      6528
          1       0.00      0.00      0.00        26

avg / total       0.99      1.00      0.99      6554

aid_related
             precision    recall  f1-score   support

          0       0.73      0.87      0.80      3832
          1       0.75      0.54      0.63      2722

avg / total       0.74      0.74      0.73      6554

medical_help
             precision    recall  f1-sco

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


### 9. Export your model as a pickle file

In [25]:
import pickle
filename = 'classifier.pkl'
pickle.dump(pipeline, open(filename, 'wb'))

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.